# Developer's guide to DashT wxWebView based instruments 

It all started from this question: what in the earth can we do with this? :
```
    wxWebView           *m_pWebPanel;
````

## Introduction 

xxx

## Scope 

xxxx

## Target environment

xxxxxx

### Operating System

zzzzz

### OpenCPN

zzzzzz

### Dashboard-Tactics plug-in

zzzzzz

#### Plug-in framework services

There are many issues with various configurations present in selected targets. It is clear that preferred protocol would be _http://_ or even _https://_, but if _file://_ is to be used because people do not want to use a HTTP server or they do not know how to set it up, the limitations and somewhat erratic way of interpreting the RFCs of the various backends is causing serious issues, now and certainly in the future!

>Some catastrophe scenarios could be, for example: Windows dropping IE being available for back-end; Linux moving a the same time to wxWidgets 3.1 (which is good) but also to WebKit 2.x; there must be others...

Result observed without no tweaking in the policies of the out-of-the-box installation:


| Platform | wxWidgets | back-end | file:// | http:// | viewport prop. font sizing |
| ---------| --------- | -------- | ------- | ------- | :------------------------: |
| Windows | 3.1 | IE(_max: 8_) | cookie: Y | cookie: Y | Y |
|  |  |  | localStorage: N | localStorage: Y | _CSS.supports()_: N (but **works**!) |
| Linux | 3.0 | WebKit1x | cookie: N | cookie: Y | Y |
|  |  |  | localStorage: Y | localStorage: Y | |
| Mac | 3.1(?) | (Mac?)WebKit(?)x | cookie: ? | cookie: ? | ? |
|  |  |  | localStorage: ? | localStorage: ? |  |

Where there is no issues to obtain persistency when the protocol is _http://_ it looks like that one needs to use cookies in Windows and _localStorage()_ elsewhere with _file://_ protocol. For Mac, we do not know enough yet. This, of course is a bit shaky for the future, even if it works now. It is better to make a parameter storage which automatically adapts ot use _localStorage()_ if it is available. This way, plan B can be that people just install that HTTP server and they get the persistent parameters back in case there is an issue with their back-end, which can change.

> Of course, one can imagine to integrate some [hefty javascript code](https://dev.to/rumkin/how-to-create-js-webserver-without-node-js-1hic) to make a standalone server _and_ the page, but it seems, for now, overkilling. Let's start with _file://_, plan B being using _http://_ with an external HTTP server and, finally, plan C being to use a local configuration file for static configuration.

## Development Paradigm

xxxxx

## Implementation

xxxxx

### Base class

zzzzz

### Abstraction layer class

zzzzz

### Implementation class

zzzzz

### Implementation application 

zzzz

## HTML/CSS/JS application development

The development cycle is a typical one for such an application. However, from the [development paradigm](#Development+Paradigm) follows that one needs always think and continously test on the final target, _i.e._ the worst case real-life environment and - if possible - make the application to automatically adapt to the conditions found on the target system and the intended usage on it.

### Snippets and ideas

You may have your own cloud based favorite to develop your HTML/CSS/JS snippets. If not, why not try https://codepen.io, some [snippets from the author](https://codepen.io/petrim) are public, of course, and can be forked. Do not hesitate to browse the featured projects of this or any other similar site!

Of course, if you are more discrete and/or hate the cloud, you can use workspaces provided by the browser based tools (see below). Not sure, though about the obtained level of secrecy and the need to have it in this case... Use what you like the best!

### Browser based development and testing

Using modern browser's `Ctrl+Shift+I` which opens the integrated tools for the web developers remains the ideal tool to debug your HTML/CSS/JS application. Nothing beats the possibility to set breakpoints, study the document structure, and to see what CSS sentences are, actually ignored by the browser and which one are doing what you expected them to do.

Because the "_browser_" compatibility - or rather the incompatibility is a real issue in this development, you can see that the `onload` event handler takes the usage of the `CSS.support()` function - when it is available! (Which we first need to detect.)

### Verifying (often) on the target system

Careful out there! Fancy design may work on you browser but does it work on `wxWidgets` `WebView` on WebKit/IE back-end?

It is a good idea to keep open and **regularly** reload your project on the `WebView` based simple browser, called - surprisingly - **_webview_**. If that browser is located on the most modest of the targeted platforms, like Raspberry Pi, that's even better.

## Modular development and packaging

`node.js`, `npm` and `webpack` - well, consider `webpack` being the `CMake` equivalent of HTML/CSS/JS world! (Or worse...)

In case you wonder why there are so many folders and files in the development area, tt is better to read a [nice tutorial like this](https://medium.com/dev-bits/everything-i-know-about-writing-modular-javascript-applications-37c125d8eddf) about this dodgy subject with many opinions; the discussion is quite demanding in number of paragraphs...

### Multi-language support

Complicated in JavaScript in general and `webpack` does not help in that. I found this example project: https://github.com/donaldpipowitch/webpack-i18n-example

### Character-set specific issues

`git` under different systems may change the encoding which can create an issue in the following case:

>Special characters as symbols must match the target systems - Javascript does not particularly know about the UTF-8, it just puts out the characters to the browser. For example, degree character ° in `common.js` is nerve-racking! This is how I managed to get it work in `justgage` `symbol` character: I used `emacs` on a Linux machine and entered the character with `C-x 8 RET B0 RET` which enters an UTF-8 encoded character. Tested OK. Now, syncing the Windows machine with GitHub Desktop. Surprise, the file is now encoded ANSI - one can verify this with `Notepad++` and the degree sign looks funny. **Do not touch it!**. As such, it actually works in IE based WebKit backend. You can test it with IE and it works both in `justgage` but also in the plain HTML (_i.e._ "simple") display.

### Static code check ESLint etc.

The static code check for pull request - or any commit whatsover - is done witch `Codacy.com` and not continous build checks with integrated `ESlint` in the `webpack` - I do not see the need to have both. Also, there is a problem to get the `ESlinit` configuration imported every time it changes into `Codacy.com` so I have given up and I use its on-line tool to set up the rule. You are invited to learn those chosen for this project from https://app.codacy.com/manual/petri38-github/dashboard_tactics_pi/patterns/list - they cannot be exported, unfortunately.

One can discuss about the meaningfulness of any rule, of course but in general, I expect grade `A` code, that means _zero_ static check error - with the given rules, of course. I will seriously hesitate if I get a grade `B` pull request reports from `Codacy.com`. So please iterate a few times to get the `A` grade so that time consuming discussions and work can be avoided.

#### The innerHTML rule

We follow the (somewhat old, in purpose since we are working an older back-ends) https://developer.mozilla.org/en-US/docs/Archive/B2G_OS/Security/Security_Automation

I have ported Firefox's class mentioned in the above instructions in `../src/escapeHTML.js` - use the class as `Sanitizer` - there are plenty of examples of its suggested usage in the code, `grep` them with `innerHTML`.

## WebView specific issues

Please find and collect also your findings about the wonderfully complex world of the WebView - on differen platforms having different back-ends!

#### WebView specific on Linux `__WXGTK__ && wxUSE_WEBVIEW_WEBKIT`

You will find that the on Raspberry Pi, or on any other Linux based system the `wxWidgets` environment is not the same than on your fancy laptop. You can find the _webview_ sample from the source distribution, here:
```
/usr/share/doc/wx3.0-examples/examples/
```
Use the unpack script to unpack the _webview_ example, run `make` and then run the application. Load your page on it and see if it works the same. If it does not, see below for debug instructions.

I tried to put TRUE these [settings](https://webkitgtk.org/reference/webkitgtk/2.4.10/webkitgtk-webkitwebview.html): `enable-file-access-from-file-uris` and `enable-universal-access-from-file-uris`, unfortunately they do not have any effect on the handling of `file://` URIs **if there are cookie-saving involved**: it does not work.

Let's try with this browser: `/usr/lib/arm-linux-gnueabihf/webkit2gtk-4.0 $ ./MiniBrowser --cookies-policy=always` : Proof of Concept (POC) which confirms that while the OpenCPN and wxWidgets still use WebKit1 on Linux, it would not be any better with WebKit2Gtk! (I was planning to try this parameter.)

>Both WebKit1x and WebKit2Gtk silently refuse to save cookies, if the URI is not `http://`  or `https://`, respecting the [RFC6265](https://tools.ietf.org/html/rfc6265). To maintain porting compatibility, use [LocalStorage](https://developer.mozilla.org/en-US/docs/Web/API/Window/localStorage) instead, perhaps with JSON parsing.

#### WebView specific on Mac `__WXGTK__`

`contribution welcome`: likel, is it `wxUSE_WEBVIEW_WEBKIT` or `wxUSE_WEBVIEW_WEBKIT2`?

#### WebView specific on Linux `__WXGTK__ && wxUSE_WEBVIEW__IE`

On Windows: `C:\wxWidgets-3.1.2\samples\webview\vc_mswud` (after the build). Compiled with default settings it works for the development purposes in this project without problem.

#### WebView specific on Linux __WXGTK__

`/usr/share/doc/wx3.0-examples/examples/` Used makefile OK.

## Debugging on the target system

First of all, make symbolic links from the OpenCPN plug-in installation folders into your development folder: you are going to need it since modifications by trial and error are needed when all you have is a blank screen on **_webview_** when you open your application which works damn fine on your fancy browser! You do not want to spend your life to reinstall the plug-in after every modification.

### Performance issues

Executing in a tight loop scripts on multiple windows soon brings up the CPU load!

Tools to observe this are the usual process tools, in Windows `procexp.exe` and on Linux `htop`

Small things to details can make a big difference! In the design paradigm we expect the data coming from engine, not from wind vane, for example. The CPU load should be not big since the dial is not moving so often, like the wind direction dial which is jumping back and forth. But we should not send the useless data to the dial: if it has the value, do not use heavy script execution to send the same value again! Remember, we do not send float values but strings. We make sure that we do not have more than one decimal. And if the value is the same, do not send it to the instrument!

The below process image details is from Windows when there is no filtering of repetitive data with twelve (12!) JS instruments, fed by the NMEA Simulator via Signal K delta channels via Signal K input streamer about 130 floats per second but with static data set (the peaks are InfluxDB Out flusing to a file):

![2020-01-20_dgb_procexp_12_instrucjs_clients_alpha_01.png](2020-01-20_dgb_procexp_12_instrucjs_clients_alpha_01.png) [(zoom)](img/2020-01-20_dgb_procexp_12_instrucjs_clients_alpha_01.png)

That is huge waste of the CPU power since the dials are not moving at all. Let's make the following change in the script sending code and then observe again:
```
        if ( m_istate == JSI_SHOWDATA ) {
            if ( m_data != m_lastdataout ) {
                wxString javascript =
                    wxString::Format(
                        L"%s%s%s",
                        "window.iface.newdata(",
                        m_data,
                        ");");
                RunScript( javascript );
                m_lastdataout = m_data;
            } // then do not load the system with the same script execution multiple times
        } // the instrument is ready for data
```

![2020-01-20_dgb_procexp_12_instrucjs_clients_alpha_02.png](2020-01-20_dgb_procexp_12_instrucjs_clients_alpha_02.png) [(zoom)](img/2020-01-20_dgb_procexp_12_instrucjs_clients_alpha_02.png)

The CPU-load looks quite reasonable around 7 percent w/ i7 CPU and the number of I/O operations is less than half of what it used to be!

Of course, when one creates heavy oscillation in the r.p.m. values, for example, the CPU load goes up, since the SVG-rendering of several dials in this case is entering into the game - there is always a price to pay for jumping dials! But it is noteworthy that the number of I/O operations remain low, nevertheless.

![2020-01-20_dgb_procexp_12_instrucjs_clients_alpha_02_cont_changes.png](2020-01-20_dgb_procexp_12_instrucjs_clients_alpha_02_cont_changes.png) [(zoom)](img/2020-01-20_dgb_procexp_12_instrucjs_clients_alpha_02_cont_changes.png)

The background load can be further reduced by **randomizing the threads** which are driving the JS instruments:

![2020-01-22_dgb_procexp_12_instrucjs_clients_alpha_02_randomized_threads.png](2020-01-22_dgb_procexp_12_instrucjs_clients_alpha_02_randomized_threads.png) [(zoom)](img/2020-01-22_dgb_procexp_12_instrucjs_clients_alpha_02_randomized_threads.png)

#### Help! My fancy application looks rotten / does not work on RPI

Let's consider our design constraints first: we are developing for cross-platform, limited or old-fashioned web application run-time support environment with very small screen or canvas area.

Sounds familiar? Yes, it is like developing a HTML/CSS/JS application for a telephone!

You need to have the patience - and more importantly - the possibility to carefully check and adjust at pixel level CSS files and debug stubborningly failing JavaScript which worked fine on your browser.

Unfortunately the _iOS_ or _Android_ USB-based development tool connections to your _Safari_ or _Chrome_ will not be any good in this case.

There is, however a solution. It is not developed any more (since 2017) but it still exists and **it still works**: [WEINRE](https://people.apache.org/~pmuellr/weinre/docs/latest/). Let's hope it is not going away, or that other tools for `wxWidgets` `WebView` would emerge!

>**Attention vulnerabilities** - _weinre_ module has its development stopped and it is as such a very vulnerable product. Use `npm audit` to see the details of those. Make sure to use --save-dev switch when installing to use it only for occasional debugging during the development phase.

While waiting the new tools, I archived [this paper](pdf/debugging_mobile_javascript_with_WEINRE_ibm_blog_2011.pdf) ([2011](https://www.ibm.com/developerworks/community/blogs/94e7fded-7162-445e-8ceb-97a2140866a9/entry/debugging_mobile_javascript_with_weinre?lang=en)) which nicely explains the remote portable device debugging concept and what you can expect and what you cannot expect from WEINRE.

##### Installating WEINRE on RPI

To use WEINRE, one needs to have web server. Luckily, on RPI this is easy:

###### Installing http-server on RPI

If you are reading this, it is highly likely that you are using the excellent [SignalK node server](https://github.com/SignalK/signalk-server-node/blob/master/raspberry_pi_installation.md) on your RPI. That means that you have `node.js` and, with it `npm` package manager.

Install [http-server](https://www.npmjs.com/package/http-server) with command `sudo npm install http-server -g`.

Using the command line, move to the directory where your application file root is located and give command `http-server`. Leave it running _et voilà_, you have a web server!

##### Get WEINRE

`sudo npm intall --save-dev weinre`

##### Configuring WEINRE

In your home folder, create a file `~/.weinre/server.properties` with the following contents:

```
boundHost:    -all-
httpPort:     8081
reuseAddr:    true
readTimeout:  1
deathTimeout: 5
```

##### Launching WEINRE server

Using (another shell) command line, type `weinre` and leave it running.

Now you should have two servers, `http-server` and `weinre` running.

##### Opening the WEINRE debugger

Start the RPI's browser, probably _Chromium_ (but I am using _Vivaldi_) and navigate to `localhost:8081`.

The server's welcome page, _weinre - web inspector remote_ will open. It will give you interesting information and even demos you may want to try first opening them on a **separate** screen or tab.

Likewise, you would open the debugger service, `localhost:8081/client/#anonymous` on a separate screen or tab. This window is now waiting for a connection from your remote (or local) `wxWidgets` `WebView` based application.

##### Preparing your application for remote debugging

With "_remote_" we understand your debug server running on the Raspberry Pi, and either a _webview_ browser or OpenCPN with its `WebView` class based instruments running on your Windows, Mac or Linux development system

Of course, it is not so "_remote_" if the main debugging target being the _webview_ browser or OpenCPN running on Raspberry Pi. But since WEINRE access event those applications through the http-server, it does not make any difference, in fact:

One can have multiple hosts used at the same time, for testing and debugging the same application on different run-time platforms.

Each instance of the application must know where WEINRE server is located. So you need to know your Raspberry Pi's IP-address in your network. Let's say it is 192.168.8.103 : In really "_remote_" test environments you would add the following line in your HTML:

```
<!--script src="http://192.168.8.103:8081/target/target-script-min.js#anonymous"></script-->
```

It is not mandatory to use it in exclusively "_local_" development, you can set:

```
<!--script src="http://127.0.0.1:8081/target/target-script-min.js#anonymous"></script-->
```

Open the your application's HTML-file in _webview_ browser - drag and drop works.

As you can see above, this makes the application to download a javascript module from the WEINRE server. It connects you the WEINRE server's debug environment. If you not see the connection under the _Remote_ tab, reload the page:

![2019-12-25_weinre_server_connected_to_local_host_client.png](2019-12-25_weinre_server_connected_to_local_host_client.png) [(zoom)](pdf/2019-12-25_weinre_server_connected_to_local_host_client.png)

##### Debugging with WEINRE

Please remember that this is **not** full blown JavaScript debugger like the one which you can find from your browser. Before coming here you have debugged your algorithms and you will use WEINRE only to find and sort out the snagging incompatibility issues between the various run-time environments.

What can one do with WEINRE, then?

###### Inspect the document structure

In the case of it is your JavaScript which dynamically constructs your document structure contents, it would be a good omen for the rest of the session if you can find actually the intended identifiers in the structure:

![2019-12-25_weinre_server_element_inspection.png](2019-12-25_weinre_server_element_inspection.png) [(zoom)](img/2019-12-25_weinre_server_element_inspection.png)

Like with the modern browsers, you can select a structure in the "_Elements_" window and it will be highlighted in the _webview_ browser, which is quite handy sometimes.

"_Resources_", "_Network_" and "_Timeline_" tabs are quite useless and you should not expect to see anything interesting there.

"_Console_", as the name implies allows you to inspect the variables and show the `console.log()` output.

![2019-12-25_weinre_server_consolelog_code.png](2019-12-25_weinre_server_consolelog_code.png) [(zoom)](img/2019-12-25_weinre_server_consolelog_code.png)

Finally, the answer to the question "_what to do?_" if your screen remains blank or if the mouse event does not work as you expect is the following: Use the good old "_comment out suspicious code blocks until your application loads_"-method!. Then put `console.log()` calls in critical points.

![2019-12-25_weinre_server_console_mouse_event.png](2019-12-25_weinre_server_console_mouse_event.png) [(zoom)](img/2019-12-25_weinre_server_console_mouse_event.png)

![2019-12-25_weinre_server_console_mouse_event_log.png](2019-12-25_weinre_server_console_mouse_event_log.png) [(zoom)](img/2019-12-25_weinre_server_console_mouse_event_log.png)

Rudimentary? Yes, but supposing that your algorithms are already tested elsewhere, WEINRE allows you to have the necessary tools to avoid guesswork based development!